# OpenAR Vision Bridge — Virtual 128×64 OLED HUD

This notebook runs YOLO on your webcam feed, then renders a low-res **128×64 monochrome HUD** to simulate an Arduino-style OLED display (SSD1306).  
Press **q** to quit.


In [ ]:
# If you haven't installed dependencies yet, uncomment and run:
# !pip install opencv-python ultralytics numpy


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# 1) Load YOLO model (Nano for speed)
# First run may download weights automatically.
model = YOLO("yolov8n.pt")

# 2) Initialize webcam
cap = cv2.VideoCapture(0)

# 3) OLED display constants (typical SSD1306)
OLED_WIDTH = 128
OLED_HEIGHT = 64
SCALE_FACTOR = 6  # enlarge the OLED on your PC screen

# Font settings for the tiny screen
font = cv2.FONT_HERSHEY_SIMPLEX

print("Starting OpenAR Simulation... Press 'q' to quit.")


In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- STEP 1: Computer Vision backend ---
    results = model(frame, stream=True, verbose=False)

    # Choose best detection (highest confidence) for a clean HUD
    detection_text = "SCANNING..."
    conf_score = ""

    for result in results:
        boxes = result.boxes
        if boxes is not None and len(boxes) > 0:
            best_box = max(boxes, key=lambda x: float(x.conf[0]))

            cls = int(best_box.cls[0])
            label = model.names[cls]
            conf = float(best_box.conf[0])

            detection_text = label.upper()
            conf_score = f"{int(conf * 100)}%"

            # Draw a box on the webcam feed (reference only)
            x1, y1, x2, y2 = map(int, best_box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # --- STEP 2: Bridge to 128×64 monochrome display ---
    oled_sim = np.zeros((OLED_HEIGHT, OLED_WIDTH), dtype=np.uint8)

    # Header bar (white)
    cv2.rectangle(oled_sim, (0, 0), (OLED_WIDTH, 12), 255, -1)
    # Black text on white bar
    cv2.putText(oled_sim, "OPEN-AR HUD", (2, 9), font, 0.3, 0, 1)

    # Main detection text
    cv2.putText(oled_sim, "TARGET:", (5, 25), font, 0.3, 255, 1)
    cv2.putText(oled_sim, detection_text, (5, 45), font, 0.6, 255, 1)

    # Confidence in corner
    if conf_score:
        cv2.putText(oled_sim, conf_score, (90, 60), font, 0.3, 255, 1)

    # Simple reticle
    cx, cy = OLED_WIDTH // 2, OLED_HEIGHT // 2
    cv2.line(oled_sim, (cx - 5, cy), (cx + 5, cy), 255, 1)
    cv2.line(oled_sim, (cx, cy - 5), (cx, cy + 5), 255, 1)

    # --- STEP 3: Visualisation (upscale to see pixels) ---
    oled_display_scaled = cv2.resize(
        oled_sim,
        (OLED_WIDTH * SCALE_FACTOR, OLED_HEIGHT * SCALE_FACTOR),
        interpolation=cv2.INTER_NEAREST,
    )

    # Show both views
    cv2.imshow("Computer Vision (Backend)", frame)
    cv2.imshow("OpenAR Glasses View (128x64)", oled_display_scaled)

    # Quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
